# 用Cython包装C++代码

Cython最大的作用其实是作为C++代码和python代码的桥梁,比如我们已经有一个C++写的程序了,但我们希望让python可以调用它,传统的做法是使用ctypes或者cffi作为桥,但这种方式需要有相当的C/C++知识.Cython的话基本可以无痛进行C++代码的包装,我们通过包装一个例子来看看cython是如何包装c++代码的


Cython现在原生的支持大多数的C++语法.
尤其是:
现在可以使用`new`和`del`关键字动态分配C++对象.

+ C++对象可以进行堆栈分配
+ C++类可以使用新的关键字`cppclass`声明。
+ 支持模板化类和函数。
+ 支持重载函数。
+ 支持C++操作符(例如operator +,operator [],...)的重载。

## 封装步骤

封装C++的步骤大致有如下几步:

1. 在setup.py脚本中或在源文件中本地指定C ++语言。
2. 使用`cdef extern from C++头文件`创建一个或多个`.pxd`文件.在pxd文件中，以`cdef cppclass`来声明类并且声明公共名称(变量,方法和构造函数）
3. 通过`cimport`引入`pxd`文件，进行`pxd`的实现代码，也就是`.pyx`文件。


## 最简单的一个例子

这个例子用来介绍Cython包装C/C++代码的步骤.例子是一个长方形类,C++代码部分如下:

In [1]:
%%writefile Rectangle.h

namespace shapes {
    class Rectangle {
    public:
        int x0, y0, x1, y1;
        Rectangle();
        Rectangle(int x0, int y0, int x1, int y1);
        ~Rectangle();
        int getArea();
        void getSize(int* width, int* height);
        void move(int dx, int dy);
    };
}

Overwriting Rectangle.h


In [2]:
%%writefile Rectangle.cpp
#include "Rectangle.h"

namespace shapes {

  Rectangle::Rectangle() { }

    Rectangle::Rectangle(int X0, int Y0, int X1, int Y1) {
        x0 = X0;
        y0 = Y0;
        x1 = X1;
        y1 = Y1;
    }

    Rectangle::~Rectangle() { }

    int Rectangle::getArea() {
        return (x1 - x0) * (y1 - y0);
    }

    void Rectangle::getSize(int *width, int *height) {
        (*width) = x1 - x0;
        (*height) = y1 - y0;
    }

    void Rectangle::move(int dx, int dy) {
        x0 += dx;
        y0 += dy;
        x1 += dx;
        y1 += dy;
    }

}

Overwriting Rectangle.cpp


## 用于包装的pyx文件

要包装C++文件,我们得先在cython中声明出这个C++的类,在cython中申明C或者C++的内容(接口)需要使用`cdef extern from ....`这种语法(外部声明).

在

In [7]:
%%writefile rect.pyx
#cython: language_level=3
# distutils: language = c++
# distutils: sources = Rectangle.cpp


cdef extern from "Rectangle.h" namespace "shapes":
    cdef cppclass Rectangle:
        Rectangle() except +
        Rectangle(int, int, int, int) except +
        int x0, y0, x1, y1
        int getArea()
        void getSize(int* width, int* height)
        void move(int, int)
cdef class PyRectangle:
    cdef Rectangle c_rect      # hold a C++ instance which we're wrapping
    def __cinit__(self, int x0, int y0, int x1, int y1):
        self.c_rect = Rectangle(x0, y0, x1, y1)
    def get_area(self):
        return self.c_rect.getArea()
    def get_size(self):
        cdef int width, height
        self.c_rect.getSize(&width, &height)
        return width, height
    def move(self, dx, dy):
        self.c_rect.move(dx, dy)

Overwriting rect.pyx


这样，我们就完成了C++的封装。而且从Python的开发角度来看，这个扩展类型看起来和感觉就像一个本地定义的Rectangle类。
需要注意的是，如果我们需要额外的属性设置方法，可以自己再添加.

## setup.py的写法

我们的setup.py和之前差不多的写法

In [8]:
%%writefile setup.py

from distutils.core import setup
from Cython.Build import cythonize
 
setup(
    name = "rectangleapp",
    ext_modules = cythonize('*.pyx')
)

Overwriting setup.py


In [9]:
!python setup.py build_ext --inplace

Compiling rect.pyx because it changed.
[1/1] Cythonizing rect.pyx
running build_ext
building 'rect' extension
creating build
creating build/temp.macosx-10.7-x86_64-3.6
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include/python3.6m -c rect.cpp -o build/temp.macosx-10.7-x86_64-3.6/rect.o
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include/python3.6m -c Rectangle.cpp -o build/temp.macosx-10.7-x86_64-3.6/Rectangle.o
g++ -bundle -undefined dynamic_lookup -L/Users/huangsizhe/LIB/CONDA/anaconda/lib -L/Users/huangsizhe/LIB/CONDA/anaconda/lib -arch x86_64 bui

In [10]:
import rect

In [11]:
pyRect = rect.PyRectangle(100, 100, 300, 500)
width, height = pyRect.get_size()
print("size: width = %d, height = %d" % (width, height))

size: width = 200, height = 400


In [12]:
pyRect.get_area()

80000

## 预算符重载

这个例子是一个vector2d,实现了加和乘.


In [89]:
%%writefile vector2d.h

namespace algebra {
    class Vec2d {
    public:
        double x, y;
        Vec2d();
        Vec2d(double x, double y);
        ~Vec2d();
        Vec2d operator+(const Vec2d& b);
        Vec2d operator*(double k);
        
    };
}

Writing vector2d.h


In [90]:
%%writefile vector2d.cpp
#include "vector2d.h"

namespace algebra {

    Vec2d::Vec2d() {
        x=0;
        y=0;
    }

    Vec2d::Vec2d(double x, double y) {
        this->x = x;
        this->y = y;
    }

    Vec2d::~Vec2d() { }
    
    Vec2d Vec2d::operator+(const Vec2d& other){

        Vec2d r = Vec2d(this->x+other.x,this->y+other.y);
        return r;
    }
    Vec2d Vec2d::operator*(double k){
        Vec2d r = Vec2d(this->x*k,this->y*k);
        return r;
    }
}

Writing vector2d.cpp


In [95]:
%%writefile vec2d_main.cpp
#include "vector2d.h"
#include <iostream>
using algebra::Vec2d;
using std::cout;
using std::endl;
        
int main(){
    Vec2d v1 = Vec2d(2.1,2.2);
    Vec2d v2 = Vec2d(2.3,2.4);
    Vec2d v3 = v1+v2;
    cout << v3.x<<endl;
    cout << v3.y<<endl;
}

Overwriting vec2d_main.cpp


In [96]:
!g++-7 -o a.out vec2d_main.cpp vector2d.cpp 

In [97]:
!./a.out

4.4
4.6


In [111]:
%%writefile vec2d.pyx
#cython: language_level=3
# distutils: language = c++
# distutils: sources = vector2d.cpp


cdef extern from "vector2d.h" namespace "algebra":
    cdef cppclass Vec2d:
        Vec2d() except +
        Vec2d(double, double) except +
        double x, y
        Vec2d operator+(Vec2d)
        Vec2d operator*(float)

cdef class PyVec2d:
    cdef Vec2d c_vec2d      # hold a C++ instance which we're wrapping
    def __cinit__(self, float x, float y):
        self.c_vec2d = Vec2d(x, y)
    @property
    def x(self):
        return self.c_vec2d.x
    @property
    def y(self):
        return self.c_vec2d.y
    
    cpdef add(self,PyVec2d other):
        cdef Vec2d c
        c = self.c_vec2d+other.c_vec2d
        return PyVec2d(c.x,c.y)
    
    cpdef mul(self,float k):
        cdef Vec2d c
        c = self.c_vec2d*k
        return PyVec2d(c.x,c.y)
    
    def __add__(self,PyVec2d other):
        return self.add(other)
    
    def __mul__(self,float k):
        return self.mul(k)
    

Overwriting vec2d.pyx


In [112]:
%%writefile setup.py

from distutils.core import setup
from Cython.Build import cythonize
 
setup(
    name = "vec2dapp",
    ext_modules = cythonize('vec2d.pyx')
)

Overwriting setup.py


In [113]:
!python setup.py build_ext --inplace

Compiling vec2d.pyx because it changed.
[1/1] Cythonizing vec2d.pyx
running build_ext
building 'vec2d' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include/python3.6m -c vec2d.cpp -o build/temp.macosx-10.7-x86_64-3.6/vec2d.o
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include -arch x86_64 -I/Users/huangsizhe/LIB/CONDA/anaconda/include/python3.6m -c vector2d.cpp -o build/temp.macosx-10.7-x86_64-3.6/vector2d.o
g++ -bundle -undefined dynamic_lookup -L/Users/huangsizhe/LIB/CONDA/anaconda/lib -L/Users/huangsizhe/LIB/CONDA/anaconda/lib -arch x86_64 build/temp.macosx-10.7-x86_64-3.6/vec2d.o build/temp.macos

In [114]:
import vec2d

In [116]:
v1 = vec2d.PyVec2d(2.1,2.2)

In [117]:
v2 = vec2d.PyVec2d(2.3,2.4)

In [119]:
(v1+v2).x

4.399999618530273